In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
import os
import csv
from tqdm import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil
from collections import defaultdict


In [4]:

#loading question and answer embeddings of train and validation data from disk
train_embeds=pd.read_csv('re_train_embeds.csv')
validation_embeds=pd.read_csv('re_validation_embeds.csv')

In [5]:

#function to preprocess question and answer embeddings post loading from disk
def preprocess_embeds(embed):
  output=[]
  embed=embed[2:-2]
  embed=embed.replace('\n','')
  for i in embed.split(' '):
    if i!=' ' and i!='':
      output.append(i)
  return (output)

     

In [6]:
#preprocessing embeddings post loading from disk for train data
from tqdm.notebook import tqdm
tqdm.pandas()
train_embeds['question_embeds']=train_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
train_embeds['answer_embeds']=train_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

  0%|          | 0/5003 [00:00<?, ?it/s]

  0%|          | 0/5003 [00:00<?, ?it/s]

In [7]:
#preprocessing embeddings post loading from validation data
validation_embeds['question_embeds']=validation_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
validation_embeds['answer_embeds']=validation_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

  0%|          | 0/504 [00:00<?, ?it/s]

  0%|          | 0/504 [00:00<?, ?it/s]

In [8]:
#function to l2 normalize each embedding
def normalize(x):
  x=np.array(x,dtype=np.float64)
  return (x)/(np.linalg.norm(x))

In [9]:
#normalizing train question and answer embeddings
train_embeds['question_embeds_norm']=train_embeds['question_embeds'].apply(lambda x: normalize(x))
train_embeds['answer_embeds_norm']=train_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [10]:
#normalizing validation question and answer embeddings
validation_embeds['question_embeds_norm']=validation_embeds['question_embeds'].apply(lambda x: normalize(x))
validation_embeds['answer_embeds_norm']=validation_embeds['answer_embeds'].apply(lambda x: normalize(x))
     

In [11]:
#Cleaning the gpt train data
train_gpt_data=train_embeds[['short_question','short_answer','question_embeds_norm','answer_embeds_norm']].copy()
train_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
train_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...,"[-0.01421890905663768, 0.025619907450341046, 0...","[-0.007459985581738011, -0.02122592276723626, ..."
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...,"[-0.013814892581875434, 0.02668001998051444, 0...","[0.00223136463861142, -0.012820215500427647, 0..."
2,what are the dietary restrictions for celiac d...,omitting gluten from the diet is the key to co...,"[-0.014280090332285491, 0.02902447327765528, 0...","[-0.005688997413084786, -0.02194953130653521, ..."


In [12]:
#saving gpt training data to disk
train_gpt_data.to_pickle("./train_gpt_data.pkl")
     

In [13]:
#cleaning gpt validation data
validation_gpt_data=validation_embeds[['short_question','short_answer','question_embeds_norm','answer_embeds_norm']].copy()
validation_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
validation_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,do i have a yeast infection,hi this can be a vaginal fungal infection whic...,"[-0.008608112898811646, 0.025272781187763167, ...","[-0.0087260827168495, -0.025829989834866878, 0..."
1,i need to buy health insurance asap what do i ...,go to healthcare gov call my husband 407 222 9...,"[-0.0022776732616655664, 0.02531196420973276, ...","[-0.010670890831805894, 0.012798066486932491, ..."
2,i had an acute ebv antibody test done and my r...,hi yes a chronic ebv infection also can be rul...,"[-0.007839831409961463, 0.024712667389081715, ...","[-0.0025002140233858117, -0.020223583268815057..."


In [14]:
#saving the gpt validation data to disk
validation_gpt_data.to_pickle("./validation_gpt_data.pkl")

In [15]:

#loading train and validation gpt data from disk
train_gpt_data=pd.read_pickle('./train_gpt_data.pkl')
validation_gpt_data=pd.read_pickle('./validation_gpt_data.pkl')

In [16]:
train_gpt_data.shape,validation_gpt_data.shape

((5003, 4), (504, 4))

In [17]:

#displaying the train gpt data
train_gpt_data.head(5)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...,"[-0.01421890905663768, 0.025619907450341046, 0...","[-0.007459985581738011, -0.02122592276723626, ..."
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...,"[-0.013814892581875434, 0.02668001998051444, 0...","[0.00223136463861142, -0.012820215500427647, 0..."
2,what are the dietary restrictions for celiac d...,omitting gluten from the diet is the key to co...,"[-0.014280090332285491, 0.02902447327765528, 0...","[-0.005688997413084786, -0.02194953130653521, ..."
3,i have had a pneumonia shot can i get either a...,you can always catch an illness from a child b...,"[-0.0051098851649243136, 0.026251631984036797,...","[-0.0028335161187219376, -0.017787302858114903..."
4,my baby ate her on poop my baby ate poop 4 day...,hi one of ours did that she is now 47 and a mu...,"[-0.006687587964836193, 0.023149305110760594, ...","[-0.00492109446937413, -0.02058334904862069, 0..."


In [18]:
import os
import csv
from tqdm import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil
from collections import defaultdict

#creating faiss index for semantic search 
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py
qa = pd.read_pickle('./train_gpt_data.pkl')
question_bert = qa["Q_FFNN_embeds"].tolist()
answer_bert = qa["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')
answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])
answer_index.add(answer_bert)

In [21]:

#defining function to prepare the actual gpt training data by retrieving similar question and answer pairs
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py
def preparing_gpt_training_data(question,answer,question_embedding):
  topk=20
  scores,indices=answer_index.search(
                  np.array([question_embedding]).astype('float32'), topk)
  q_sub=qa.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: %s' % (
                        question, answer)
  encoded_len=len(tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return tokenizer.encode(line)[-1024:]
     

In [23]:
#loading pre-trained gpt2 tokenizer and gpt2 model
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("gpt2")

E:\anacondafiles\envs\geekl\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\manis\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [24]:
qa['gpt_data']=qa.progress_apply(lambda x: preparing_gpt_training_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

  0%|          | 0/5003 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1031 > 1024). Running this sequence through the model will result in indexing errors


In [26]:
validation_gpt_data['gpt_data']=validation_gpt_data.progress_apply(lambda x: preparing_gpt_training_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)
     

  0%|          | 0/504 [00:00<?, ?it/s]

In [27]:
#function to get the start of positive mask to be used while creating the masks
def mask_start(gpt_data):
  return 1024-gpt_data[::-1].index(4600)+1

In [28]:

#finding the mask start for the train dataset
qa['mask_start']=qa.gpt_data.progress_apply(lambda x: mask_start(x))

  0%|          | 0/5003 [00:00<?, ?it/s]

In [29]:

#finding the mask start for the validation dataset
validation_gpt_data['mask_start']=validation_gpt_data.gpt_data.progress_apply(lambda x: mask_start(x))

  0%|          | 0/504 [00:00<?, ?it/s]

In [30]:
#saving train data to disk
qa.to_pickle('./qa_gpt_inbuild.pkl')
     

In [31]:

#finding length of the gpt data for train dataset
qa['gpt_lens']=qa.gpt_data.apply(lambda x: len(x))

In [32]:
#finding length of the gpt data for validation dataset
validation_gpt_data['gpt_lens']=validation_gpt_data.gpt_data.apply(lambda x: len(x))

In [33]:

#finding number of train datapoints that has sequence length less than 1024
qa[qa.gpt_lens<1024].shape

(54, 7)

In [34]:
#finding number of validation datapoints that has sequence length less than 1024
validation_gpt_data[validation_gpt_data.gpt_lens<1024].shape
     

(1, 7)

In [35]:

#finding train datapoints that has sequence length of 1024
gpt_data_cleaned=qa[qa.gpt_lens==1024]
gpt_data_cleaned.shape

(4949, 7)

In [36]:
#finding validation datapoints that has sequence length of 1024
validation_gpt_data_cleaned=validation_gpt_data[validation_gpt_data.gpt_lens==1024]
validation_gpt_data_cleaned.shape

(503, 7)

In [37]:

#function to create the loss mask given mask start
def return_loss_mask(mask_start):
  return [0]*mask_start+[1]*(1024-mask_start)
     

In [38]:
#creating loss mask for train data
gpt_data_cleaned['mask']=gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

C:\Users\manis\AppData\Local\Temp\ipykernel_9768\2211518662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_data_cleaned['mask']=gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))


In [39]:
#creating loss mask for validation data
validation_gpt_data_cleaned['mask']=validation_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

C:\Users\manis\AppData\Local\Temp\ipykernel_9768\3417697302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_gpt_data_cleaned['mask']=validation_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))


In [40]:

#saving train gpt data with loss mask to disk
gpt_data_cleaned.to_pickle('./qa_gpt_inbuild_cleaned.pkl')

In [41]:

#saving validation gpt data with loss mask to disk
validation_gpt_data_cleaned.to_pickle('./validation_gpt_inbuild_cleaned.pkl')

In [42]:
#loading gpt train and validation data from disk
gpt_data_cleaned=pd.read_pickle('./qa_gpt_inbuild_cleaned.pkl')
validation_gpt_data_cleaned=pd.read_pickle('./validation_gpt_inbuild_cleaned.pkl')
gpt_data_cleaned.shape,validation_gpt_data_cleaned.shape

((4949, 8), (503, 8))

In [44]:
#extracting gpt data for train data
gpt_data=[]
for i in gpt_data_cleaned.gpt_data.values:
  gpt_data.append(i)
gpt_data=np.array(gpt_data)
gpt_data.shape

(4949, 1024)

In [45]:
#extracting gpt data for validation data
validation_gpt_data=[]
for i in validation_gpt_data_cleaned.gpt_data.values:
  validation_gpt_data.append(i)
validation_gpt_data=np.array(validation_gpt_data)
validation_gpt_data.shape

(503, 1024)

In [46]:
#extracting loss mask for train data
loss_mask=[]
for i in gpt_data_cleaned['mask'].values:
  loss_mask.append(i)
loss_mask=np.array(loss_mask)
loss_mask.shape
     

(4949, 1024)

In [47]:
#extracting loss mask for validation data
validation_loss_mask=[]
for i in validation_gpt_data_cleaned['mask'].values:
  validation_loss_mask.append(i)
validation_loss_mask=np.array(validation_loss_mask)
validation_loss_mask.shape

(503, 1024)

In [48]:

#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt train dataset
import tensorflow as tf
BATCH_SIZE = 1
BUFFER_SIZE = 20000
gpt_train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': gpt_data,

        'label': gpt_data[:,1:],
        'loss_mask':loss_mask[:,:-1]
    },
))


gpt_train_dataset = gpt_train_dataset.shuffle(BUFFER_SIZE)
gpt_train_dataset = gpt_train_dataset.batch(BATCH_SIZE)
gpt_train_dataset = gpt_train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
     

In [49]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt validation dataset
import tensorflow as tf
BATCH_SIZE = 1
BUFFER_SIZE = 20000
validation_gpt_train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': validation_gpt_data,

        'label': validation_gpt_data[:,1:],
        'loss_mask':validation_loss_mask[:,:-1]
    },
))

validation_gpt_train_dataset = validation_gpt_train_dataset.shuffle(BUFFER_SIZE)
validation_gpt_train_dataset = validation_gpt_train_dataset.batch(BATCH_SIZE)
validation_gpt_train_dataset = validation_gpt_train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
     

In [50]:

#displaying sample gpt training data
next(iter(gpt_train_dataset))
     

({'context': <tf.Tensor: shape=(1, 1024), dtype=int32, numpy=array([[ 423,  407, 1760, ...,  714, 1037,  517]])>,
  'label': <tf.Tensor: shape=(1, 1023), dtype=int32, numpy=array([[ 407, 1760,  616, ...,  714, 1037,  517]])>,
  'loss_mask': <tf.Tensor: shape=(1, 1023), dtype=int32, numpy=array([[0, 0, 0, ..., 1, 1, 1]])>},)

In [51]:
#training the gpt2 model
learning_rate=0.0001
optim=tf.optimizers.Adam(learning_rate)
j=0
loss_cum=0
epoch_loss=0
stop_training=False
epochs=1
for epoch in range(epochs):
  print('epoch',epoch)
  for i in tqdm(gpt_train_dataset):
    with tf.GradientTape() as tape:
      context=i[0]['context']
      loss_mask=tf.cast(i[0]['loss_mask'],tf.float32)
      output=tf_gpt2_model(context,training=True)['logits']
      loss_value = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=np.array(i[0]['label']), logits=output[ :,:-1])*loss_mask)
      j=j+1
      loss_cum+=loss_value
      epoch_loss+=loss_value/100
      if j%50==0:
        print(loss_cum/50)
        grads=tape.gradient(loss_cum/50,tf_gpt2_model.trainable_variables)
        optim.apply_gradients(zip(grads,tf_gpt2_model.trainable_variables))
        loss_cum=0
      if tf.math.is_nan(loss_value):
        print('loss became nan')
        stop_training=True
        break
      if j%10000==0:
        tf_gpt2_model.save_pretrained("tf_gpt2_model_2_10"+str(epoch)+'_'+str(j))
  print('epoch_loss by 100',epoch_loss/237)
  epoch_loss=0
  if stop_training:
    break
  tf_gpt2_model.save_pretrained("tf_gpt2_model_2_10"+str(epoch))

epoch 0


  1%|▊                                                                             | 49/4949 [04:08<6:30:11,  4.78s/it]

tf.Tensor(497.96515, shape=(), dtype=float32)


  2%|█▌                                                                            | 99/4949 [08:13<6:20:53,  4.71s/it]

tf.Tensor(384.63486, shape=(), dtype=float32)


...

tf.Tensor(355.31622, shape=(), dtype=float32)


 99%|███████████████████████████████████████████████████████████████████████████▏| 4899/4949 [6:32:53<03:53,  4.68s/it]

tf.Tensor(419.272, shape=(), dtype=float32)


100%|████████████████████████████████████████████████████████████████████████████| 4949/4949 [6:36:51<00:00,  4.81s/it]


epoch_loss by 100 tf.Tensor(80.57373, shape=(), dtype=float32)


In [52]:
"tf_gpt2_model_2_10"+str(epoch)

'tf_gpt2_model_2_100'